# Task 1

Check out this official repository with many examples of Keras implementations of various sorts of
deep neural networks here. We recommend cloning this repository and trying to get some of these
examples running on your system (or Colab/DeepNote). In particular, experiment with mnist mlp.py
and mnist cnn.py scripts which show you how to build simple neural networks for the MNIST dataset
(useful for the next task

Next, take the two well-known datasets: Fashion MNIST (introduced in Ch 10, p. 298) and CIFAR-10.
The first dataset contains 2D (grayscale) images of size 28x28, split into 10 categories; 60,000 images
for training and 10,000 for testing, while the latter contains 32x32x3 RGB images (50,000/10,000
train/test). Apply two reference networks on the fashion MNIST datase

## (a) MLP

initializations, activations, optimizers (and
their hyperparameters), regularizations (L1, L2, Dropout, no Dropout). You may also experiment
with changing the architecture of both networks: adding/removing layers, number of convolutional
filters, their sizes, etc.

In [1]:
import tensorflow as tf

2025-11-07 11:33:00.314375: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-07 11:33:00.358995: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-07 11:33:06.163251: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Check number of available GPUs

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [3]:
# Use TensorFlow's bundled Keras to ensure compatibility with GPUs
from tensorflow import keras
import os
from sklearn.preprocessing import StandardScaler

random_state = 900
keras.utils.set_random_seed(random_state)

# Try to enable memory growth for all GPUs so TF doesn't reserve all GPU memory upfront
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print('Enabled memory growth for GPUs:', gpus)
    except Exception as e:
        print('Could not set memory growth:', e)
else:
    print('No GPU devices found by TensorFlow')



fashion_mnist= keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()
print(X_train_full.shape, y_train_full.shape)

X_valid, X_train = X_train_full[:5000] / 255.0, X_train_full[5000:] / 255.0
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

X_test = X_test / 255.0
print(X_train.shape, y_train.shape)
print(X_valid.shape, y_valid.shape)

Enabled memory growth for GPUs: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
(60000, 28, 28) (60000,)
(55000, 28, 28) (55000,)
(5000, 28, 28) (5000,)


### Model 1

Below is our first attempt at building a neural network. We use the SGD optimizer.

In [21]:
## First attempt

# Build model

activation_function = 'leaky_relu'

model1 = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(400, activation=activation_function, kernel_regularizer=keras.regularizers.l2(0.0001)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(200, activation=activation_function),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(100, activation=activation_function),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(50, activation=activation_function),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(10, activation="softmax")
])
early_stopping = keras.callbacks.EarlyStopping(
    patience=6,
    restore_best_weights=True
)

model1.summary()


model1.compile(loss="sparse_categorical_crossentropy",
optimizer=keras.optimizers.SGD(learning_rate=5e-2),
metrics=["accuracy",
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ])
model1.fit(
    X_train, y_train,
    epochs=50,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping]
)


/vol/home/s2358441/.conda/envs/IDL/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_8 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (None, 400)            │       314,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_38 (Dropout)            │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (None, 200)            │        80,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_39 (Dropout)            │ (None, 200)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (None, 100)            │        20,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_40 (Dropout)            │ (None, 100)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_41 (Dropout)            │ (None, 50)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (None, 10)             │           510 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 419,860 (1.60 MB)

 Trainable params: 419,860 (1.60 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 13s 6ms/step - accuracy: 0.7843 - loss: 0.6499 - val_accuracy: 0.8576 - val_loss: 0.4584
Epoch 2/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8446 - loss: 0.4796 - val_accuracy: 0.8648 - val_loss: 0.4326
Epoch 3/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.8581 - loss: 0.4378 - val_accuracy: 0.8726 - val_loss: 0.4117
Epoch 4/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8675 - loss: 0.4115 - val_accuracy: 0.8786 - val_loss: 0.3907
Epoch 5/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8743 - loss: 0.3931 - val_accuracy: 0.8758 - val_loss: 0.3943
Epoch 6/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - accuracy: 0.8787 - loss: 0.3804 - val_accuracy: 0.8860 - val_loss: 0.3718
Epoch 7/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8831 - loss: 0.3669 - val_accuracy: 0.8830 - val_loss: 0.3779
Epoch 8/50
1719/1719 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8861 - loss: 0.3586 -

In [22]:
#evaluate the model on the test set
test_loss, test_acc = model1.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)
#0.8834999799728394

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8889 - loss: 0.3692
Test accuracy: 0.8888999819755554
Test loss: 0.36924466490745544


### Model 2

In [1]:
## second attempt

# Build model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation='leaky_relu', kernel_regularizer=keras.regularizers.l2(0.0001)),
    keras.layers.Dropout(0.1),
    keras.layers.Dense(100, activation="leaky_relu", kernel_regularizer=keras.regularizers.l2(0.0001)),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(100, activation="leaky_relu"),
    keras.layers.Dropout(0.05),
    keras.layers.Dense(30, activation="leaky_relu"),    
    keras.layers.Dense(10, activation="softmax")
])
early_stopping = keras.callbacks.EarlyStopping(
    patience=6,
    restore_best_weights=True
)

model.summary()


model.compile(loss="sparse_categorical_crossentropy",
optimizer=keras.optimizers.Adam(learning_rate=1e-4),
metrics=["accuracy",
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ])
model.fit(
    X_train, y_train,
    epochs=60,
    validation_data=(X_valid, y_valid),
    callbacks=[early_stopping]
)


NameError: name 'keras' is not defined

In [ ]:
#evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
print('Test loss:', test_loss)
#0.8866999745368958

  1/313 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.8750 - loss: 0.5421

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8867 - loss: 0.3845
Test accuracy: 0.8866999745368958
Test loss: 0.38445159792900085


## (b) CNN

Here we make a model 

In [ ]:
max_pool = keras.layers.MaxPool2D(pool_size=2)
# avg_pool = keras.layers.AveragePooling2D(pool_size=2)
model = keras.models.Sequential([
keras.layers.Conv2D(64, 7, activation="leaky_relu", padding="same",
input_shape=[28, 28, 1]),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(2),
keras.layers.Conv2D(128, 3, activation="leaky_relu", padding="same"),
keras.layers.BatchNormalization(),
keras.layers.Conv2D(128, 3, activation="leaky_relu", padding="same"),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(2),
# keras.layers.Dropout(0.12,
keras.layers.Conv2D(256, 3, activation="leaky_relu", padding="same"),
keras.layers.BatchNormalization(),
keras.layers.Conv2D(256, 3, activation="leaky_relu", padding="same"),
keras.layers.BatchNormalization(),
keras.layers.MaxPooling2D(2),
keras.layers.Flatten(),
keras.layers.Dense(128, activation="leaky_relu"),
keras.layers.Dropout(0.5),
keras.layers.Dense(64, activation="leaky_relu"),
keras.layers.Dropout(0.5),
keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy",
optimizer=keras.optimizers.SGD(learning_rate=2e-3),
metrics=["accuracy",
        #   tf.keras.metrics.Precision(), tf.keras.metrics.Recall()
          ])
model.fit(
    X_train, y_train,
    epochs=30,
    validation_data=(X_valid, y_valid),
    # callbacks=[early_stopping]
)
#evaluate the model on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)
#base:0.8420000076293945
#leaky: 0.8525000214576721
#leaky + L2regularization: 0.8472999930381775
#leaky + batch normalization: 0.8978000283241272

(print(tf.__version__))
# metrics=["accuracy"])

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)